# Week 11 Problem 1

A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says YOUR CODE HERE. Do not write your answer in anywhere else other than where it says YOUR CODE HERE. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select Kernel, and restart the kernel and run all cells (Restart & Run all).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select File → Save and CheckPoint)

5. When you are ready to submit your assignment, go to Dashboard → Assignments and click the Submit button. Your work is not submitted until you click Submit.

6. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

7. If your code does not pass the unit tests, it will not pass the autograder.

## Author: Apurv Garg
### Primary Reviewer: Radhir Kothuri


# Due Date: 6 PM, April 09, 2018

In [3]:
# Import email library, policy controls how to process the data
import email as em
from email import policy
import numpy as np
import os
from IPython.display import HTML
from numpy.testing import assert_array_equal, assert_array_almost_equal
from pandas.util.testing import assert_frame_equal, assert_index_equal
from nose.tools import assert_false, assert_equal, assert_almost_equal, assert_true, assert_in, assert_is_not
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier

## Datset(Problem1)

For this problem, we will be reading an email from the directory and will find out the main attributes of the email message.

In [4]:
with open('/home/data_scientist/data/misc/INFO490.eml') as fin:
    msg = em.message_from_file(fin, policy=policy.default)

# Problem 1

For this problem, complete the function `vals` which will take `message` as parameter and return the receiver's information(whom the message was sent to), email subject, email date, email content type, email content language, html payload, plain text payload content type and html payload content type in that order. <br>
For last 3 return values, check if the email has multiple parts, after which extract different parts (or payloads) from the message.

In [5]:
def vals(message):
    '''           
    Parameters
    ----------
    message : email message
    
    Returns
    -------
    A tuple of 8 containing receiver's information(whom the message was sent to), 
    email subject, 
    email date, 
    email content type, 
    email content language, 
    html payload, 
    plain text payload content type, 
    html payload content type
    
    HINT
    ----
    'Content-Language' gives the language of the email content
    To find different payload content type, explore different get methods
    '''    
    # YOUR CODE HERE
    to = msg['to']
    sub = msg['subject']
    date = msg['date']
    con_tp = msg['content-type']
    con_lang = msg['content-language']
    
    if msg.is_multipart:
#         data = msg.get_payload(0).get_content()
        data = msg.get_payload(0)
        html = msg.get_payload(1)
        data_tp = data.get_content_type()
        html_pl = html.get_content_type()
        
    return(to, sub, date, con_tp, con_lang, html, data_tp, html_pl)

In [6]:
to, subject, date, content_type, language, html, data_content, html_content = vals(msg)
assert_equal(to, '"Garg, Apurv" <apurvg2@illinois.edu>')
assert_equal(subject, 'INFO490')
assert_equal(date, 'Sat, 24 Mar 2018 23:02:35 -0500')
assert_equal(subject, 'INFO490')
assert_equal(content_type, 'multipart/alternative; boundary="B_3604777369_1001855260"')
assert_equal(language, 'en-US')
assert_equal(data_content, 'text/plain')
assert_equal(html_content, 'text/html')


In [7]:
HTML(html.get_content())


## Dataset(Problem 2 and 3)

For the next 2 problems, we will be classifying the mails into spam and ham. Though Naive Bayes' is the most commonly used technique for the spam-ham classification, we will be trying out RandomForest and SGDClassifier in the next 2 problems.

In [12]:
# First we find our HOME directory
home_dir = !echo $HOME

# Define data directory
home = home_dir[0] +'/data/'
mypath = home + 'email/'

ham = []
spam = []

# Max number of files to read
max_files = 500

# Read in good (ham) emails
for root, dirs, files in os.walk(os.path.join(mypath, 'ham')):
    for count, file in enumerate(files):
    
        # To control memory usage, we limit the number of files
        if count >= max_files:
            break
            
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)

            ham.append(data.decode(encoding='ISO-8859-1'))

# Read in bad (spam) emails
for root, dirs, files in os.walk(os.path.join(mypath, 'spam')):
    for count, file in enumerate(files):
        
        # To control memory usage, we limit the number of files
        if count >= max_files:
            break
           
        with open(os.path.join(root, file), encoding='ISO-8859-1') as fin:
            msg = em.message_from_file(fin, policy=policy.default)
            for part in msg.walk():
                if part.get_content_type() == 'text/plain':
                    data = part.get_payload(None, decode=True)

            spam.append(data.decode(encoding='ISO-8859-1'))

# For text analysis, we need NumPy arrays.
# Convert the text lists to NumPy arrays

pos_emails = np.array(ham)
neg_emails = np.array(spam) 

# Create label arrays
pos_labels = np.ones(pos_emails.shape[0])
neg_labels = np.zeros(neg_emails.shape[0])
# We split positive/negative emails into two groups test/train each. 
# This value must be less than max_files.
split_value = 100

# We combine neg and positive into four arrays.
x_train = np.concatenate((pos_emails[:split_value], 
                          neg_emails[:split_value]), axis = 0)

x_test = np.concatenate((pos_emails[split_value:],
                         neg_emails[split_value:]), axis = 0)

y_train = np.concatenate((pos_labels[:split_value], 
                          neg_labels[:split_value]), axis = 0)

y_test = np.concatenate((pos_labels[split_value:],
                         neg_labels[split_value:]), axis = 0)


# Problem 2

For this problem, complete the function `rf_class` which will take `x_train`, `y_train` and `x_test` as parameters and return the pipeline object(with a TfidfVectorizer and RandomForestClassifier) and predicted value for test set.

- The parameters to be used for the TfidfVectorizer are `stop_words`='english', `ngram_range`=(2,2), `lowercase`=True and `min_df`=2. Other parameters should be kept as default. 
- Create a RandomForestClassifier with parameters: `min_samples_split`=5 and `random_state`=40. Other parameters are left to be default. <br>
- Fit estimator to training data after vectorizing and predict for the test dataset using this model.


In [23]:
def rf_class(x_train, y_train, x_test):
    '''           
    Parameters
    ----------
    x_train : training set features
    y_train : training set labels
    x_test : test set features
    
    Returns
    -------
    A tuple of 2 containing the pipeline object(with a TfidfVectorizer and RandomForestClassifier), 
    predicted value for test set 
    ''' 
    # YOUR CODE HERE
    steps = [('TfidfVectorizer',TfidfVectorizer(stop_words='english', ngram_range=(2,2), min_df=2)),\
             ('RandomForestClassifier', RandomForestClassifier(min_samples_split=5, random_state=40))]
             
    pipeline = Pipeline(steps)
             
    pipeline.fit(x_train, y_train)
    pred = pipeline.predict(x_test)         
             
    return pipeline, pred

In [24]:
pipeline1, y_pred1 = rf_class(x_train, y_train, x_test)
val1, val2 = pipeline1.named_steps.items()
tf = val1[0]
rf = val2[0]
assert_equal(isinstance(pipeline1, Pipeline), True)
assert_equal(isinstance(pipeline1.named_steps[rf], RandomForestClassifier), True)
assert_equal(isinstance(pipeline1.named_steps[tf], TfidfVectorizer), True)
assert_equal(pipeline1.named_steps[tf].get_params()['stop_words'], 'english')
assert_equal(pipeline1.named_steps[tf].get_params()['ngram_range'], (2,2))
assert_equal(pipeline1.named_steps[tf].get_params()['lowercase'], True)
assert_equal(pipeline1.named_steps[tf].get_params()['min_df'], 2)
assert_equal(pipeline1.named_steps[rf].get_params()['min_samples_split'], 5)
assert_almost_equal(100.0 * metrics.accuracy_score(y_test, y_pred1), 97.625, 1)

In [25]:
print(metrics.classification_report(y_test, y_pred1, 
                                    target_names = ['Ham', 'Spam']))

             precision    recall  f1-score   support

        Ham       0.96      0.99      0.98       400
       Spam       0.99      0.96      0.98       400

avg / total       0.98      0.98      0.98       800



# Problem 3

For this problem, complete the function `sgd_class` which will take `x_train`, `y_train` and `x_test` as parameters and return the pipeline object(with a CountVectorizer and SGDClassifier) and predicted value for test set.

- The parameters to be used for the CountVectorizer are `stop_words`='english', `ngram_range`=(1,3) and `lowercase`=True. Other parameters should be kept as default. 
- Create a SGDClassifier with parameters: `penalty`='l1' and `random_state`=40. Other parameters are left to be default. <br>
- Fit estimator to training data after vectorizing and predict for the test dataset using this model.


In [26]:
def sgd_class(x_train, y_train, x_test):
    '''           
    Parameters
    ----------
    x_train : training set features
    y_train : training set labels
    x_test : test set features
    
    Returns
    -------
    A tuple of 2 containing the pipeline object(with a CountVectorizer and SGDClassifier), 
    predicted value for test set 
    '''    
    # YOUR CODE HERE
    pipeline = Pipeline([('CountVectorizer',CountVectorizer(stop_words='english',\
                        ngram_range=(1,3), lowercase=True)), ('SGDClassifier', \
                        SGDClassifier(penalty='l1', random_state=40))])
    pipeline.fit(x_train, y_train)
    
    pred = pipeline.predict(x_test)
    
    return pipeline, pred
    

In [27]:
pipeline2, y_pred2 = sgd_class(x_train, y_train, x_test)
val1, val2 = pipeline2.named_steps.items()
cv = val1[0]
sgd = val2[0]
assert_equal(isinstance(pipeline2, Pipeline), True)
assert_equal(isinstance(pipeline2.named_steps[cv], CountVectorizer), True)
assert_equal(isinstance(pipeline2.named_steps[sgd], SGDClassifier), True)
assert_equal(pipeline2.named_steps[cv].get_params()['stop_words'], 'english')
assert_equal(pipeline2.named_steps[cv].get_params()['ngram_range'], (1,3))
assert_equal(pipeline2.named_steps[cv].get_params()['lowercase'], True)
assert_equal(pipeline2.named_steps[sgd].get_params()['penalty'], 'l1')
assert_almost_equal(100.0 * metrics.accuracy_score(y_test, y_pred2), 91.000000, 1)

In [28]:
print(metrics.classification_report(y_test, y_pred2, 
                                    target_names = ['Ham', 'Spam']))

             precision    recall  f1-score   support

        Ham       0.99      0.83      0.90       400
       Spam       0.85      0.99      0.92       400

avg / total       0.92      0.91      0.91       800

